In [1]:
import requests, json, csv, os
from datetime import datetime
import time

In [2]:
global begin, end

In [3]:
users = {}
reject_user = {}

In [4]:
champion = {}
items = {}

In [5]:
challengerEntries = {}
grandEntries = {}
masterEntries = {}

In [6]:
matches = {}

In [52]:
by_match = {}

In [12]:
apikey = ""
region = "br1.api.riotgames.com"

In [8]:
def getReq(api_url):
    response = requests.get(api_url)
    if(response.status_code == 200):
        return response.status_code, response.json()
    else:
        return response.status_code, None

In [13]:
def getUser(id):
    for u in users:
        if id == users[u]['id'] or id in reject_user:
            print(id + " signed in")
            return False
    api_url = "https://" + region + "/lol/summoner/v4/summoners/" + id + "?api_key=" + apikey
    code, text = getReq(api_url)
    if code == 200:
        print(id + " sign in")
        users[text['id']] = text
        return True
    else:
        print(id + " _ _ _ " + str(code))
        if code == 429:
            time.sleep(30)
            return getUser(id)
        elif code == 404:
            reject_user[id] = id
            return False
        else:
            return code

In [50]:
def getByMatch(user, id):
    try:
        if by_match[user]:
            try:
                if by_match[user][id]:
                    print(id + " match in" + user)
                    return False
            except:
                print(id + " not in " + user)
    except:
            print(user + " not in match")
    url = "https://americas.api.riotgames.com/lol/match/v5/matches/" + id + "?api_key=" + apikey
    print(id)
    code, text = getReq(url)
    if code == 200:
        print(id + " get Match")
        by_match[user] = {}
        by_match[user][id] = text
        return True
    else:
        print(id + " _ _ _ " + str(code))
        if code == 429:
            time.sleep(30)
            return getByMatch(user, id)
        elif code == 404:
            return False
        else:
            return code
     

In [15]:
by_match

{}

In [16]:
def getMatches(id):
    if users[id]['puuid'] in matches:
        print(id + " has matches")
        return False
    url = "https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/" + users[id]['puuid'] + "/ids?type=ranked&start=0&count=10&api_key=" + apikey
    code, text = getReq(url)
    if code == 200:
        print(id + " get Match")
        matches[users[u]['puuid']] = text
        return True
    else:
        print(id + " _ _ _ " + str(code))
        if code == 429:
            time.sleep(30)
            return getMatches(id)
        elif code == 404:
            return False
        else:
            return code

In [17]:
def writeCSV(setData):
    # now we will open a file for writing
    data_file = open(setData + '.csv', 'w')
    
    # create the csv writer object
    csv_writer = csv.writer(data_file)
    
    # Counter variable used for writing
    # headers to the CSV file
    count = 0
    isKeyed = False
    setVariable = globals()[setData]
    for key in setVariable:
        #print(key)
        #print(setVariable[key])
        if count == 0:
    
            # Writing headers of CSV file
            data_header = setVariable[key]
            #if ("key" or "id") in list(data_header.keys()):
            if "key" in list(data_header.keys()) or "id" in list(data_header.keys()):
                header = data_header.keys()                
            else:
                isKeyed = True
                header = ['id']
                for x in list(data_header.keys()):
                    header.append(x)           
            csv_writer.writerow(header)
            count += 1
    
        # Writing data of CSV file
        if isKeyed:
            values = [key]
            for x in list(setVariable[key].values()):
                values.append(x)
            csv_writer.writerow(values)
        else:
            csv_writer.writerow(setVariable[key].values())

    data_file.close()

In [18]:
def writeCSV2(setData):
     # now we will open a file for writing
    data_file = open(setData + '.csv', 'w')
    
    # create the csv writer object
    csv_writer = csv.writer(data_file)
    
    # Counter variable used for writing
    # headers to the CSV file
    setVariable = globals()[setData]
    header = []
    for x in list(setVariable[0].keys()):
        header.append(x)   
    print(header)   
    csv_writer.writerow(header)
    for key in setVariable:
        values = []
        for x in list(key.values()):
            values.append(x)
        csv_writer.writerow(values)

    data_file.close()

In [ ]:
tags = "champion", "item"
for tag in tags:
    url = "http://ddragon.leagueoflegends.com/cdn/12.9.1/data/en_US/" + tag + ".json"
    globals()[tag] = getReq(url)['data']

    writeCSV(tag)

In [24]:
url = "https://br1.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=" + apikey
begin = datetime.now()
code, text = getReq(url)
if code == 200:
    count = 1
    masterEntries = text['entries']
    #masterEntries.sort(key=lambda item: item.get("summonerId"))
    #print(masterEntries)
    for i, master in enumerate(masterEntries):
        id = master['summonerId']
        dec = float(i)/float(len(masterEntries))*100.00
        dec = "{:.2f}".format(dec)
        print(str(i) + "/" + str(len(masterEntries)) + " - " + dec + "%")
        get = getUser(id)
        if get != True:
            print(id, "error " + str(get))
        end = datetime.now()
        if i%30 == 0 or (end - begin).seconds == 10:
            time.sleep(15)
            begin = datetime.now()
else:
    print(code)


0/3743 - 0.00%
E1BgoaJE-cvcWm3HWYT_dq-xrS7Psoo2paWSuNZRP57BWw sign in
1/3743 - 0.03%
QgqWL8B-se6cV8Jbglb2W11AmleWzJpGgdV3Xgu9JGzCyv0 sign in
2/3743 - 0.05%
7spZd-dIctt_xQdNDOzi3P5U6ZBEGfqaFB4S9e7yBcokZg sign in
3/3743 - 0.08%
x_ECGmrETv0SYdK2tk7TGJmbFda4ZEMunrs_B_AFmHn3dQ sign in
4/3743 - 0.11%
DhOouRF5y5cVRPJJdX6B6KQgzI9qcWhfPDkBYfYE0ksTMMs sign in
5/3743 - 0.13%
KShncvVxl7t31urdEP6y_CKCXDySqFFSYoMFXcV-5Uw8dLI sign in
6/3743 - 0.16%
Jp-G-qexICAYsE-0QUJ-Y3ejjOM4oKAd96LXMX908tYwtw sign in
7/3743 - 0.19%
g12KOHTRbin0DdigNXpgtpS4pQaEX_g1od2fyPGGxQ_r sign in
8/3743 - 0.21%
NvoHP0P6aujNX5G-7G7OqE7mfqSJkazDnNDNpBKpq8zrwR4 sign in
9/3743 - 0.24%
O3fCV6g2FkYGXPv-WF-ofAdn8HFMKTqtLTs6gvG6maVnEQ sign in
10/3743 - 0.27%
UNN4S0xkg9YeY8Ahio2Umu4u01XIguiiIJfJXPrbS2g6rvY sign in
11/3743 - 0.29%
0sIqIdPDg5tsW_tvgOmczwpMDQVQu7mvUtJtIwKKHs9wFQ sign in
12/3743 - 0.32%
aDNeHV8qs7Vl4dnHCHZ5b-BeULcRgcifzDrtr5QhUl-MsktXM8tj090XEA sign in
13/3743 - 0.35%
--jyvj6oPtTGPcw1WelNZKPI3g-xEQ0qiToVrS2Ze-RYTR0 sign in


In [25]:
url = "https://br1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=" + apikey
begin = datetime.now()
code, text = getReq(url)
if code == 200:
    count = 1
    challengerEntries = text['entries']
    #masterEntries.sort(key=lambda item: item.get("summonerId"))
    #print(masterEntries)
    for i, entry in enumerate(challengerEntries):
        id = entry['summonerId']
        dec = float(i)/float(len(challengerEntries))*100.00
        dec = "{:.2f}".format(dec)
        print(dec + "%")
        get = getUser(id)
        if get != True:
            print(id, "error " + str(get))
        end = datetime.now()
        if i%30 == 0 or (end - begin).seconds == 10:
            time.sleep(15)
            begin = datetime.now()
else:
    print(code)


0.00%
m01kP7V0lYiZVCpDsJxniN-WZJy7SepnhFM9_BggkBnkH2o sign in
0.50%
yepLUlxfoPkhhj_6IsiegPjbM5T_qUnmFjl05FOHcY-rX1E sign in
1.00%
CGwJ2OiLo_Aa2WMs7WTVhij56NcIqdRCuG0FaPxPTS8fQD8 sign in
1.50%
E-8R2nvN1ZlSziqRlgaHgQegMRvLoKv_ZGME0fU-AS3U_w sign in
2.00%
hy8bfaAp3z-9ljwrTOLLuHtP7EHktMt8UxfQOs8Ynlw6elhUNYN5d_dQzA sign in
2.50%
BGdj5cYvr6r4aPik63AC0oTzG05d4usFFq1ATpQy7hOmAO9iedG6yo6JSg sign in
3.00%
Z6F_L7pKv3oH7sTLV8AmrnX7aWMkKIIZMimPCFY4BHJUlQ sign in
3.50%
OiYJGSmxoUvRP9se2DHA99aVbk4DcKGxHqUkKW7UF2Lfv-cmGqBd7BCIIA sign in
4.00%
XMcUQOTtfUADkbQ4RGQoQ1jhfTi-tJ2KM43JgMU1_7ZOTA sign in
4.50%
wFQvHK7X-TRAYjqAwDKfHDVmHU0HZtqbelIBDGLMzja38sJt_WQGJKv22w sign in
5.00%
K3jGW4zSYdF4r1KSAVzS4GS3k_efpy7NyYIO8NDi9IMV7gs sign in
5.50%
ID9QXX-tJPwkBUwdCuI5TzTjHiYu00oIK4zc5w808Brj sign in
6.00%
_HzYiTz01GpZ4Qdxol7HniT2QZINGo9Se2gIhUlNLXuD5gI sign in
6.50%
ZoNVy5Z_rI9Muz918w3r-6YUZ4hGd-ow2iwetuNd24NN sign in
7.00%
6SppPDU9NvPOc0cu2kTSrKqt3Dl75Ev8htPQ7ZPc3BedIw sign in
7.50%
Pzgadc8PeePTHlV-5e2WjRkKMYUyqJ

In [26]:
url = "https://br1.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=" + apikey
begin = datetime.now()
code, text = getReq(url)
if code == 200:
    count = 1
    grandEntries = text['entries']
    #masterEntries.sort(key=lambda item: item.get("summonerId"))
    print(len(grandEntries))
    for i, entry in enumerate(grandEntries):
        id = entry['summonerId']
        dec = float(i)/float(len(grandEntries))*100.00
        dec = "{:.2f}".format(dec)
        print(dec + "%")
        get = getUser(id)
        if get != True:
            print(id, "error " + str(get))
        end = datetime.now()
        if i%30 == 0 or (end - begin).seconds == 10:
            time.sleep(15)
            begin = datetime.now()
else:
    print(code)

500
0.00%
luw8ZnkuqWWK9RoedhG7XlP-nICzX8-08wB-zrbwCxh_wm0 sign in
0.20%
KwYn6jwasbJ82Wt1Q1HaqakqP1LNZGc4dNF3nf6m4VAf83AOVSjaknJOiA sign in
0.40%
blbE62lzQOyN20Qnkxd5IXzgePXzr2xsEooFcW8GtxwjZTI sign in
0.60%
G9Ji9uJNCmxhbYm_1OON3IqfDgoxFIaZJTH6fUjgYGMz2xE sign in
0.80%
VN-aEx7OiLkcWKcKs66UO9yplAsnzujV7fZclJM3KtqAgdAfDZ9l1K8Yng sign in
1.00%
AIOmxkUmaZj_7SapvyNI6FLUnr8HUaanG8U84uOseF0xug sign in
1.20%
B1tYAZpAXnlvHCZwvcMuYv2czQeoBZzgFmpj0mh0bfMnsKY sign in
1.40%
SEtjM2lF2KFagvcOJxT1bOZTfADapykIFa6F--sxOM5yw5A sign in
1.60%
ZiZOKK0DXzQ1T0j0WY982rPDGfRiWioRj3v-q8skT1Lc sign in
1.80%
MpPKAH6YnigIO89N4WHdYhcoTKiMkcY2NzqWkOyV4cZrkA sign in
2.00%
jIS7_2sDDEh0E5CTV0ftxbSvO6HaKUlaMzFX4KVyhqqcNwULOL0-VnXahA sign in
2.20%
hYVsmmHkDD-yADtdltKIfbZN3Dbqs8D9BA_c7EurfW3LyQ sign in
2.40%
LfAehq8cv6ApbWt2gq0ckYQHFSZBaYnqgwEBvdpSWEombQ sign in
2.60%
SGjPBlRfVuLoB6vF8OiDZ3NqbZFaEKo4aCCK2Jva3wN1qA sign in
2.80%
JsYbiiyKb0iADV0CRaSO9O62U9tE3VLPSOPe3HUhp5Ayjg sign in
3.00%
eLkcgbCcdB3ZdtY89E-V4joFQMBaXmiRXfsN

In [27]:
'''writeCSV2("challengerEntries")
writeCSV2("masterEntries")
writeCSV2("grandEntries")
'''

'writeCSV2("challengerEntries")\nwriteCSV2("masterEntries")\nwriteCSV2("grandEntries")\n'

In [32]:
begin = datetime.now()
for i, u in enumerate(users):
    dec = float(i)/float(len(users))*100.00
    dec = "{:.2f}".format(dec)
    print(str(i) + "/"+ str(len(users)) + " - " + dec + "%")
    get = getMatches(u)
    if get != True:
        print(id, "error " + str(get))
    end = datetime.now()
    print(str((end - begin).seconds) + " seconds")
    if i%500 == 0 or (end - begin).seconds >= 10:
        print("STOP")
        time.sleep(15)
        begin = datetime.now()

matches


0/4445 - 0.00%
7TKsH8qZPkMJvxPc0reKClpqW2cT4LGY7lwCoA3XDoyiSw has matches
C2z-wMH3tJncXm4d6zYfI1ym37X1IQLQrRTAIHXGDP5egw error False
0 seconds
STOP
1/4445 - 0.02%
ISLemxRCqbPqwyYvBgA3XWBAhWcBKeJxPFUSHLjUnB5IyQ has matches
C2z-wMH3tJncXm4d6zYfI1ym37X1IQLQrRTAIHXGDP5egw error False
0 seconds
2/4445 - 0.04%
cAMprL84ino7loJlF9_OYrdcFRKjjL_CnrBcnwbPMmc_KZY has matches
C2z-wMH3tJncXm4d6zYfI1ym37X1IQLQrRTAIHXGDP5egw error False
0 seconds
3/4445 - 0.07%
LMOX2DqYs3twz4wRhw0fj7m2I6WsuK7_uh9AM05m4kdFQQ has matches
C2z-wMH3tJncXm4d6zYfI1ym37X1IQLQrRTAIHXGDP5egw error False
0 seconds
4/4445 - 0.09%
GG2aY7Xs4k2kmDXZ91CHCMn3ntD5gInbOD3AkcRWc6eMvpslTAKR4W3UGQ has matches
C2z-wMH3tJncXm4d6zYfI1ym37X1IQLQrRTAIHXGDP5egw error False
0 seconds
5/4445 - 0.11%
MRNieS0sqUIHzSKssNaVXF8YfQdukJHpqCJWpzvTNyh0oNk has matches
C2z-wMH3tJncXm4d6zYfI1ym37X1IQLQrRTAIHXGDP5egw error False
0 seconds
6/4445 - 0.13%
jYXV9uDXxnNAyHRYvCj8BqtCUJC9jgV9oWxRH9DjWzZrdhuuBwbjeEWCyQ has matches
C2z-wMH3tJncXm4d6zYfI1ym37X1IQLQrRTAI

{'QmCg7c1QVsP8n2mK47oM8FnCQbqCDGdrZtgfUKKHuyfSizb_tlf0ppXBKJTGb35f7cau0-HtpKo0xQ': ['BR1_2524847284',
  'BR1_2524794309',
  'BR1_2524726652',
  'BR1_2524770852',
  'BR1_2524707849',
  'BR1_2523470714',
  'BR1_2523416840',
  'BR1_2523401168',
  'BR1_2523327779',
  'BR1_2523352811'],
 'RrmLWGWHEGNUNBJTOHmil1Czx0OIahU8eDOjdRm-JWg9vMv8EB2mKKxOPmwZo0s62oASSM_hczu8Dg': ['BR1_2524991018',
  'BR1_2524949825',
  'BR1_2523902026',
  'BR1_2523736897',
  'BR1_2523725378',
  'BR1_2523720179',
  'BR1_2523618122',
  'BR1_2523636176',
  'BR1_2523654747',
  'BR1_2522549204'],
 'MeqG_JF3hSguyawSTyrR5MYXVfcUvprwqZCkd-a4XEsRxNad8SMWVIVrskfsGE7Gqwh8q-jYaUbbig': ['BR1_2522340853',
  'BR1_2522327075',
  'BR1_2522316329',
  'BR1_2522315755',
  'BR1_2522315144',
  'BR1_2522303161',
  'BR1_2522282787',
  'BR1_2522311941',
  'BR1_2522281293',
  'BR1_2522259205'],
 'glSCmwTHcX1HHMf8X4wgKqU8wN6s7Jh_IjEyrOr8R8hp4Qq0-0Q4kX9LQzJirkQcSOJk4v9Q6-zwog': ['BR1_2525136267',
  'BR1_2525153138',
  'BR1_2525079456',
  'BR1_25

In [53]:
total_matches = len(matches)*10
begin = datetime.now()
for j, m in enumerate(matches):
    print(m)
    for i, match in enumerate(matches[m]):
        indice = i+10*j
        dec = float(indice)/float(total_matches)*100.00
        dec = "{:.3f}".format(dec)
        print(str(indice) + "/"+ str(total_matches) + " - " + dec + "%")
        get = getByMatch(m, match)
        if get != True:
            print(id, "error " + str(get))
        end = datetime.now()
        if indice%250 == 0 or (end - begin).seconds == 10:
            time.sleep(15)
            begin = datetime.now()
        


QmCg7c1QVsP8n2mK47oM8FnCQbqCDGdrZtgfUKKHuyfSizb_tlf0ppXBKJTGb35f7cau0-HtpKo0xQ
0/44450 - 0.000%
QmCg7c1QVsP8n2mK47oM8FnCQbqCDGdrZtgfUKKHuyfSizb_tlf0ppXBKJTGb35f7cau0-HtpKo0xQ not in match
BR1_2524847284
BR1_2524847284 get Match
1/44450 - 0.002%
BR1_2524794309 not inQmCg7c1QVsP8n2mK47oM8FnCQbqCDGdrZtgfUKKHuyfSizb_tlf0ppXBKJTGb35f7cau0-HtpKo0xQ
BR1_2524794309
BR1_2524794309 get Match
2/44450 - 0.004%
BR1_2524726652 not inQmCg7c1QVsP8n2mK47oM8FnCQbqCDGdrZtgfUKKHuyfSizb_tlf0ppXBKJTGb35f7cau0-HtpKo0xQ
BR1_2524726652
BR1_2524726652 get Match
3/44450 - 0.007%
BR1_2524770852 not inQmCg7c1QVsP8n2mK47oM8FnCQbqCDGdrZtgfUKKHuyfSizb_tlf0ppXBKJTGb35f7cau0-HtpKo0xQ
BR1_2524770852
BR1_2524770852 get Match
4/44450 - 0.009%
BR1_2524707849 not inQmCg7c1QVsP8n2mK47oM8FnCQbqCDGdrZtgfUKKHuyfSizb_tlf0ppXBKJTGb35f7cau0-HtpKo0xQ
BR1_2524707849
BR1_2524707849 get Match
5/44450 - 0.011%
BR1_2523470714 not inQmCg7c1QVsP8n2mK47oM8FnCQbqCDGdrZtgfUKKHuyfSizb_tlf0ppXBKJTGb35f7cau0-HtpKo0xQ
BR1_2523470714
BR1_25234707

ConnectionError: HTTPSConnectionPool(host='americas.api.riotgames.com', port=443): Max retries exceeded with url: /lol/match/v5/matches/BR1_2524949825?api_key=RGAPI-238f3d73-4293-4025-971d-20a43fe3b68d (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x10501ba00>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [ ]:
by_match

{}